# Análisis Exploratorio de Datos | Lluvias en Australia

El dataset elegido contiene datos de aproximadamente 10 años de observaciones meteorológicas diarias de muchos lugares de Australia.

### Listado de variables y selección

#### VARIABLES DE ENTRADA

#### Utilizadas

* Date: fecha de observación.
* Location: nombre común de la ubicación de la estación meteorológica.
* MinTemp: temperatura mínima en grados centígrados.
* MaxTemp: temperatura máxima en grados centígrados.
* Rainfall: la cantidad de lluvia registrada para el día en mm.
* Evaporation: la llamada evaporación de la bandeja de clase A (mm) en las 24 horas anteriores a las 9 am.
* Sunshine: número de horas de sol brillante en el día.
* WindGustDir: dirección de la ráfaga de viento más fuerte en las 24 horas anteriores a la medianoche.
* WindGustSpeed: velocidad (km/h) de la ráfaga de viento más fuerte en las 24 horas anteriores a la medianoche.
* WinDir9am: dirección del viento a las 9 am.
* WindDir3pm: dirección del viento a las 3 pm.
* WinSpeed9am: velocidad del viento (km/h) promediada durante 10 minutos antes de las 9 am.
* WinSpeed3pm: velocidad del viento (km/h) promediada durante 10 minutos antes de las 3 pm.
* Humidity9am: humedad (porcentaje) a las 9 am.
* Humidity3pm: humedad (porcentaje) a las 3 pm.
* Pressure9am: presión atmosférica (hpa) reducida al nivel medio del mar a las 9 am.
* Pressure3pm: presión atmosférica (hpa) reducida al nivel medio del mar a las 3 pm.
* Cloud9am: fracción del cielo oscurecida por las nubes a las 9 am. Esto se mide en “octas”, que son una unidad de octavos. Registra cuántos octavos del cielo están oscurecidos por las nubes. Una medida 0 indica cielo completamente despejado, mientras que un 8 indica que está completamente nublado.
* Cloud3pm: fracción de cielo oscurecida por nubes (en “octas”: octavos) a las 3 pm.
* Temp9am: temperatura (grados centígrados) a las 9 am.
* Temp3pm: temperatura (grados centígrados) a las 3 pm.
* RainToday: booleano. 1 si la precipitación (mm) en las 24 horas anteriores a las 9 am excede 1 mm, de lo contrario 0.

#### No utilizadas

#### VARIABLE DE SALIDA

* RainTomorrow: la cantidad de lluvia del día siguiente (mm) utilizada para crear la variable de respuesta RainTomorrow. Una especie de medida del “riesgo”. Valores posibles: Yes, No.

### Análisis detallado de un conjunto de variables